In [1]:
import os
import os.path as osp
import cv2
import time
import sys

import argparse
import mmcv
import torch
import torch.distributed as dist
import torch.multiprocessing as mp

import torch
import torch.nn.functional as F
import logging
import os
import os.path as osp
import cv2
from tqdm import tqdm
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
import os
import torch
""
try:
    from mmcv.utils import Config, DictAction
except:
    from mmengine import Config, DictAction

In [4]:

import torch
import torch.nn.functional as F
import logging
import os
import os.path as osp

#os.system('nvidia-smi')

# import cupy

import sys
# CODE_SPACE=os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
sys.path.append("hf_build/Metric3D/mono/")
try:
    from mmcv.utils import Config, DictAction
except:
    from mmengine import Config, DictAction
from mono.utils.logger import setup_logger
import glob
from mono.model.monodepth_model import get_configured_monodepth_model
from mono.utils.running import load_ckpt
from mono.utils.do_test import transform_test_data_scalecano, get_prediction
from mono.utils.custom_data import load_from_annos, load_data

from mono.utils.avg_meter import MetricAverageMeter
from mono.utils.visualization import save_val_imgs, create_html, save_raw_imgs, save_normal_val_imgs
import cv2
from tqdm import tqdm
import numpy as np
from PIL import Image, ExifTags
import matplotlib.pyplot as plt

from mono.utils.unproj_pcd import reconstruct_pcd, save_point_cloud
from mono.utils.transform import gray_to_colormap
from mono.utils.visualization import vis_surface_normal

In [ ]:
cfg_large = Config.fromfile('./mono/configs/HourglassDecoder/vit.raft5.large.py')
model_large = get_configured_monodepth_model(cfg_large, )
model_large, _,  _, _ = load_ckpt('./weight/metric_depth_vit_large_800k.pth', model_large, strict_match=False)
model_large.eval()

device = "cuda"
model_large.to(device)
# model_small.to(device)


In [6]:
cfg_large = Config.fromfile('./mono/configs/HourglassDecoder/vit.raft5.large.py')
fx=1000
fy=1000
img=Image.open("IMG20240610231100.jpg")
cv_image = np.array(img) 
img = cv2.cvtColor(cv_image, cv2.COLOR_BGR2RGB)
intrinsic = [fx, fy, img.shape[1]/2, img.shape[0]/2]
rgb_input, cam_models_stacks, pad, label_scale_factor = transform_test_data_scalecano(img, intrinsic, cfg_large.data_basic)

In [7]:
rgb_input

tensor([[[ 0.0056,  0.0056,  0.0056,  ...,  0.0056,  0.0056,  0.0056],
         [ 0.0056,  0.0056,  0.0056,  ...,  0.0056,  0.0056,  0.0056],
         [ 0.0056,  0.0056,  0.0056,  ...,  0.0056,  0.0056,  0.0056],
         ...,
         [ 0.0056,  0.0056,  0.0056,  ...,  0.0056,  0.0056,  0.0056],
         [ 0.0056,  0.0056,  0.0056,  ...,  0.0056,  0.0056,  0.0056],
         [ 0.0056,  0.0056,  0.0056,  ...,  0.0056,  0.0056,  0.0056]],

        [[-0.0049, -0.0049, -0.0049,  ..., -0.0049, -0.0049, -0.0049],
         [-0.0049, -0.0049, -0.0049,  ..., -0.0049, -0.0049, -0.0049],
         [-0.0049, -0.0049, -0.0049,  ..., -0.0049, -0.0049, -0.0049],
         ...,
         [-0.0049, -0.0049, -0.0049,  ..., -0.0049, -0.0049, -0.0049],
         [-0.0049, -0.0049, -0.0049,  ..., -0.0049, -0.0049, -0.0049],
         [-0.0049, -0.0049, -0.0049,  ..., -0.0049, -0.0049, -0.0049]],

        [[ 0.0082,  0.0082,  0.0082,  ...,  0.0082,  0.0082,  0.0082],
         [ 0.0082,  0.0082,  0.0082,  ...,  0

In [9]:
cam_models_stacks.shape

AttributeError: 'list' object has no attribute 'shape'